In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
// import $ivy.`sh.almond::almond-spark:0.3.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()

// val spark = AmmoniteSparkSession
//     .builder()
//     .master("local[*]")
//     .getOrCreate()

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  
// import $ivy.`sh.almond::almond-spark:0.3.0`


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@331bc294

In [2]:
def sc = spark.sparkContext

val rdd = sc.parallelize(1 to 100000000, 100)

val n = rdd.map(x => x + 1).reduce((x, y) => x + y)


// 1 2 3 4 5
// reduce(f)
// f(1, 2) f(3, 4) 5
// 3 7 5
// f(3, 7) 5
// 10 5
// f(10, 5)


reduce at cmd1.sc:5

100 / 100

defined function sc
rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at cmd1.sc:3
n: Int = 1087459712

In [3]:
val lines = sc.textFile("data/crime.csv")

val lineLengths = lines.map(s => s.length)

val totalLength = lineLengths.reduce((a, b) => a + b)

reduce at cmd2.sc:5

2 / 2

lines: org.apache.spark.rdd.RDD[String] = data/crime.csv MapPartitionsRDD[3] at textFile at cmd2.sc:1
lineLengths: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[4] at map at cmd2.sc:3
totalLength: Int = 58903091

In [4]:
val lines = sc.textFile("data/crime.csv").cache()

val totalLength = lines
    .map(s => s.length)
    .reduce((a, b) => a + b)

val counts = lines
    .flatMap(x => x.split(","))
    .map(s => (s, 1))     // RDD[(key, value)]
    .reduceByKey((a, b) => a + b)
    .sortBy(x => -x._2)
    .take(10)

reduce at cmd3.sc:5

2 / 2

map at cmd3.sc:9

2 / 2

sortBy at cmd3.sc:11

...

sortBy at cmd3.sc:11

2 / 2

take at cmd3.sc:12

1 / 1

lines: org.apache.spark.rdd.RDD[String] = data/crime.csv MapPartitionsRDD[6] at textFile at cmd3.sc:1
totalLength: Int = 58903091
counts: Array[(String, Int)] = Array(
  ("", 380873),
  ("Part Three", 162928),
  ("2017", 100938),
  ("Part Two", 100283),
  ("2016", 99134),
  ("2018", 74356),
  ("Part One", 63231),
  ("2015", 53392),
  ("B2", 51288),
  ("Friday", 49758)
)

In [5]:
val crimeFacts = spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/crime.csv")

crimeFacts.show

csv at cmd4.sc:5

1 / 1

csv at cmd4.sc:5

4 / 4

show at cmd4.sc:7

1 / 1

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|          STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+----------------+-----------+------------+--------------------+
|     I182080058|        2403|  Disorderly Conduct|DISTURBING THE PEACE|     E18|           495|    null|2018-10-03 20:13:00|2018|   10|  Wednesday|  20|  Part Two|    ARLINGTON ST|42.26260773|-71.12118637|(42.26260773, -71...|
|     I182080053|        3201|       Property Lost|     PROPERTY - LOST|     D14|       

crimeFacts: DataFrame = [INCIDENT_NUMBER: string, OFFENSE_CODE: int ... 15 more fields]

In [6]:
crimeFacts.printSchema

root
 |-- INCIDENT_NUMBER: string (nullable = true)
 |-- OFFENSE_CODE: integer (nullable = true)
 |-- OFFENSE_CODE_GROUP: string (nullable = true)
 |-- OFFENSE_DESCRIPTION: string (nullable = true)
 |-- DISTRICT: string (nullable = true)
 |-- REPORTING_AREA: string (nullable = true)
 |-- SHOOTING: string (nullable = true)
 |-- OCCURRED_ON_DATE: timestamp (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- HOUR: integer (nullable = true)
 |-- UCR_PART: string (nullable = true)
 |-- STREET: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Location: string (nullable = true)



In [7]:
import spark.implicits._

$"OFFENSE_CODE" + 1
"OFFENSE_CODE" + 1
// crimeFacts.select($"OFFENSE_CODE" + 1)
// crimeFacts.select("OFFENSE_CODE" + 1)

import spark.implicits._


res6_1: Column = (OFFENSE_CODE + 1)
res6_2: String = "OFFENSE_CODE1"

In [8]:
crimeFacts.createOrReplaceTempView("crimes")

In [9]:
spark.sql("select INCIDENT_NUMBER, DISTRICT from crimes limit 10").show

show at cmd8.sc:1

1 / 1

+---------------+--------+
|INCIDENT_NUMBER|DISTRICT|
+---------------+--------+
|     I182080058|     E18|
|     I182080053|     D14|
|     I182080052|      B2|
|     I182080051|      A1|
|     I182080050|      A7|
|     I182080049|     C11|
|     I182080048|    null|
|     I182080047|      B2|
|     I182080045|     E18|
|     I182080044|      D4|
+---------------+--------+



In [11]:
import spark.implicits._

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Crime (
    INCIDENT_NUMBER: String,
    OFFENSE_CODE: Int,
    OFFENSE_CODE_GROUP: String,
    OFFENSE_DESCRIPTION: String,
    DISTRICT: String,
    REPORTING_AREA: String,
    SHOOTING: String,
    OCCURRED_ON_DATE: String,
    YEAR: Int,
    MONTH: Int,
    DAY_OF_WEEK: String,
    HOUR: Int,
    UCR_PART: String,
    STREET: String,
    Lat: Double,
    Long: Double,
    Location: String
) {
    
    def wasShooting: Boolean = { SHOOTING != "null" }
    
}

crimeFacts.as[Crime].filter(x => x.OFFENSE_DESCRIPTION == "VANDALISM").show(3)
crimeFacts.as[Crime].filter(x => x.wasShooting).show(3)

show at cmd10.sc:29

1 / 1

+---------------+------------+------------------+-------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+--------+--------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|OFFENSE_CODE_GROUP|OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|UCR_PART|        STREET|        Lat|        Long|            Location|
+---------------+------------+------------------+-------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+--------+--------------+-----------+------------+--------------------+
|     I182080049|        1402|         Vandalism|          VANDALISM|     C11|           351|    null|2018-10-02 20:40:00|2018|   10|    Tuesday|  20|Part Two|DORCHESTER AVE|42.29919694|-71.06046974|(42.29919694, -71...|
|     I182080016|        1402|         Vandalism|          VANDALISM|     C11|           405|    null|2018-09-30 00:

show at cmd10.sc:30

1 / 1

+---------------+------------+------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|      STREET|        Lat|        Long|            Location|
+---------------+------------+------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+------------+-----------+------------+--------------------+
|     I182080058|        2403|Disorderly Conduct|DISTURBING THE PEACE|     E18|           495|    null|2018-10-03 20:13:00|2018|   10|  Wednesday|  20|  Part Two|ARLINGTON ST|42.26260773|-71.12118637|(42.26260773, -71...|
|     I182080053|        3201|     Property Lost|     PROPERTY - LOST|     D14|           795|    null|2018-08-3

import spark.implicits._


defined class Crime

In [ ]:
val offenseCodes = spark.read.option("header", "true").option("inferSchema", "true").csv("data/offense_codes.csv")
offenseCodes.show(false)

In [ ]:
import spark.implicits._

crimeFacts
    .join(offenseCodes, $"CODE" === $"OFFENSE_CODE")
    .where($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    .show(false)

In [ ]:
import org.apache.spark.sql.functions.broadcast

val offenseCodesBroadcast = broadcast(offenseCodes)

val robberyStatsWithBroadcast = crimeFacts
    .join(offenseCodesBroadcast, $"CODE" === $"OFFENSE_CODE")
    .filter($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    
robberyStatsWithBroadcast.show